In [27]:
import pandas as pd
import random
from numpy import NaN
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.preprocessing import Imputer 
from sklearn.ensemble import RandomForestRegressor 
from sklearn import model_selection
from sklearn import metrics
import textblob
import pickle
from sklearn.externals import joblib
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

# cleaning data

In [115]:
#cleaning input data to get predictor values and target
def prepare_data(fname):
    filename = os.path.join('/Users/rithika/Documents/247ai/datasets', fname)
    if os.path.isfile(filename):
        dat = pd.read_csv(filename)
    else:
        print("no such file exists at this time")
    dat['Adcopy']=dat['Headline 1']+' '+dat['Headline 2']+' '+dat['Description']
    #dat['Adcopy'] = dat['Description']
    dat = dat[['Adcopy','Clicks']]
    dat = dat[pd.notnull(dat['Adcopy'])]
    dat['Adcopy'] = dat['Adcopy'].replace('http\S+|www.\S+', '', regex=True)
    dat['Adcopy'].replace('[™!®"#\'©()*+,-./:;<=>?@\&[\]^_`{|}~’”“′‘\\\%0123456789£]',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('   ',' ',inplace=True,regex=True)
    dat['Adcopy'].replace('  ',' ',inplace=True,regex=True)
    dat['Adcopy'] = dat['Adcopy'].str.lower()
    dat['Adcopy'] = dat['Adcopy'].apply(lambda x: ' '.join([wordnet_lemmatizer.lemmatize(str(word)) for word in x.split()]))
    dat['Copy'] = dat['Adcopy'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    data = dat[['Copy','Clicks']]
    datum = data.groupby('Copy')['Clicks'].mean().reset_index()
    print(len(datum))
    return datum

In [117]:
data = prepare_data('Digitonomy_2018-06-01_2018-06-30.csv')

17074


In [118]:
#getting unique values 
datum = data.groupby('Copy')['Clicks'].mean().reset_index()
cleaned_data = datum['Copy'].tolist() #predictor values
result = datum['Clicks'] #target

In [120]:
len(cleaned_data)

17074

In [121]:
len(result)

17074

# creating BOW with TF-IDF vectors

In [122]:
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vec_train = vectorizer.fit_transform(cleaned_data)
#Save vectorizer.vocabulary_
pickle.dump(vectorizer,open("feature_bow.pkl","wb"))
# encode document
#vector = vectorizer.transform(cleaned_data)
X = np.array(vec_train.toarray())
y = np.array(result)

In [123]:
# summarize
print(vectorizer.vocabulary_)
#print(vectorizer.idf_)

{'acceptance': 2, 'loan': 640, 'apply': 52, 'free': 443, 'clever': 226, 'instant': 540, 'decision': 297, 'month': 744, 'rep': 1009, 'apr': 59, 'mobile': 732, 'access': 4, 'fast': 401, 'money': 737, 'average': 78, 'second': 1069, 'simply': 1098, 'online': 829, 'today': 1253, 'choose': 221, 'term': 1232, 'borrow': 135, 'fuss': 455, 'top': 1260, 'asset': 71, 'funding': 453, 'get': 464, 'finance': 407, 'quote': 972, 'fund': 451, 'industry': 535, 'competitive': 250, 'business': 162, 'deal': 294, 'financing': 410, 'quickly': 966, 'compare': 244, 'low': 669, 'rate': 981, 'best': 104, 'action': 8, 'credit': 279, 'history': 508, 'considered': 254, 'simple': 1097, 'application': 51, 'form': 436, 'help': 499, 'find': 412, 'right': 1030, 'accepted': 3, 'people': 878, 'would': 1377, 'recommend': 990, 'high': 504, 'approval': 56, 'ever': 377, 'advance': 12, 'approved': 57, 'advent': 13, 'uk': 1293, 'minute': 727, 'cash': 185, 'sent': 1080, 'min': 719, 'quick': 960, 'adverse': 14, 'score': 1057, 'wel

In [124]:
# summarize encoded vector
print(vec_train.shape)
print(vec_train.toarray())

(17074, 1389)
[[0.         0.         0.68512373 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.78649157]
 [0.         0.         0.         ... 0.         0.         0.69032737]
 [0.         0.         0.         ... 0.         0.         0.69443987]]


In [125]:
X.shape

(17074, 1389)

In [126]:
y.shape

(17074,)

# RF Regressor with cross validation

In [ ]:
#kfold
kf = model_selection.KFold(n_splits=5) 
kf.get_n_splits(X)
print(kf)

In [110]:
#creating instance of RFRegressor 
model1 = RandomForestRegressor(n_estimators=5000, max_features=0.6, n_jobs=-1 ,min_samples_leaf=30)

In [111]:
#cross val split with score and r2
for train_index, test_index in kf.split(X):
    #print('TRAIN:', train_index, 'TEST:', test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [112]:
#training
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.6, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=30, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [113]:
#evaluating
y_pred = model1.predict(X_test)

In [114]:
print('MSE')
print(metrics.mean_squared_error(y_test, y_pred)) #mean square error
print('R2')
print(metrics.r2_score(y_test, y_pred)) #r2 score

MSE
20.034633691035527
R2
-0.0020303682299496195


In [68]:
from scipy.stats import spearmanr, pearsonr

In [69]:
#monotonic relationship as the relation between the variables is non linear
spearman = spearmanr(y_test, y_pred)
print(f'Test data Spearman correlation: {spearman[0]:.3}')

Test data Spearman correlation: 0.184


In [38]:
#cross val score
#scores = model_selection.cross_val_score(model1, X, y, cv = 5, scoring ='neg_mean_squared_error')
#print(scores)

# predict for unseen data 

In [39]:
unseen_data = 'get instant decision'
unseen_data1 = 'decision get instant'

In [40]:
#Loading it later and creating vector of unseen data
loaded_vec = pickle.load(open("feature_bow.pkl", "rb"))
new_vec = loaded_vec.transform(np.array([unseen_data]))

In [41]:
pred = model1.predict(new_vec)

In [42]:
pred.tolist()

[0.28538813284335446]

In [43]:
new_vec1 = vectorizer.transform(np.array([unseen_data1]))

In [44]:
pred1 = model1.predict(new_vec1)

In [45]:
pred1.tolist()

[0.2853881328433544]